You can order print and ebook versions of *Think Python 3e* from
[Bookshop.org](https://bookshop.org/a/98697/9781098155438) and
[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325).

# Functions and Interfaces

This chapter introduces a module called `jupyturtle`, which allows you to create simple drawings by giving instructions to an imaginary turtle.
We will use this module to write functions that draw squares, polygons, and circles -- and to demonstrate **interface design**, which is a way of designing functions that work together.

## The jupyturtle module

To use the `jupyturtle` module, we can import it like this.

In [1]:
from jupyturtle import *

Now we can use the functions defined in the module, like `make_turtle` and `forward`.

In [2]:
t = make_turtle()
t.forward(100)

`make_turtle` creates a **canvas**, which is a space on the screen where we can draw, and a turtle, which is represented by a circular shell and a triangular head.
The circle shows the location of the turtle and the triangle indicates the direction it is facing.

`forward` moves the turtle a given distance in the direction it's facing, drawing a line segment along the way.
The distance is in arbitrary units -- the actual size depends on your computer's screen.

We will use functions defined in the `jupyturtle` module many times, so it would be nice if we did not have to write the name of the module every time.
That's possible if we import the module like this.

In [3]:
import jupyturtle as jt

This version of the import statement imports `make_turtle` and `forward` from the `jupyturtle` module so we can call them like this.

In [4]:
t = jt.make_turtle()
t.forward(100)

`jupyturtle` provides two other functions we'll use, called `left` and `right`.
We'll import them like this.

In [7]:
from jupyturtle import left, right

`left` causes the turtle to turn left. It takes one argument, which is the angle of the turn in degrees.
For example, we can make a 90 degree left turn like this.

In [8]:
left(90)
right(90)

This program moves the turtle east and then north, leaving two line segments behind.
Before you go on, see if you can modify the previous program to make a square.

## Making a square

Here's one way to make a square.

In [10]:
from jupyturtle import *
t = make_turtle()
t.forward(100)
t.left(90)
t.forward(100)
t.left(90)
t.forward(100)
t.left(90)
t.forward(100)



Because this program repeats the same pair of lines four times, we can do the same thing more concisely with a `for` loop.

In [11]:
from jupyturtle import make_turtle
t = make_turtle()
for i in range(4):
    t.forward(100)
    t.left(90)

## Encapsulation and generalization

Let's take the square-drawing code from the previous section and put it in a function called `square`.

In [18]:
def square(t, size):
    for i in range(4):
        t.forward(size)
        t.left(90)

Now we can call the function like this.

In [19]:
square(t, 100)

Wrapping a piece of code up in a function is called **encapsulation**.
One of the benefits of encapsulation is that it attaches a name to the code, which serves as a kind of documentation. Another advantage is that if you re-use the code, it is more concise to call a function twice than to copy and paste the body!

In the current version, the size of the square is always `50`.
If we want to draw squares with different sizes, we can take the length of the sides as a parameter. 

In [20]:
def square(length):
    for i in range(4):
        t.forward(length)
        t.left(90)

Now we can draw squares with different sizes.

In [21]:
square(100)
square(50)

Adding a parameter to a function is called **generalization** because it makes the function more general: with the previous version, the square is always the same size; with this version it can be any size.

If we add another parameter, we can make it even more general.
The following function draws regular polygons with a given number of sides.

In [22]:
def polygon(t, length, n):
    angle = 360 / n
    for i in range(n):
        t.forward(length)
        t.left(angle)

In a regular polygon with `n` sides, the angle between adjacent sides is `360 / n` degrees. 

The following example draws a `7`-sided polygon with side length `30`.

In [24]:
def polygon(t, length, n):
    angle = 360 / n
    for i in range(n):
        t.forward(length)
        t.left(angle)

polygon(t, 30, 7)

When a function has more than a few numeric arguments, it is easy to forget what they are, or what order they should be in. 
It can be a good idea to include the names of the parameters in the argument list.

In [26]:
polygon(t, length=30, n=7)

These are sometimes called "named arguments" because they include the parameter names.
But in Python they are more often called **keyword arguments** (not to be confused with Python keywords like `for` and `def`).

This use of the assignment operator, `=`, is a reminder about how arguments and parameters work -- when you call a function, the arguments are assigned to the parameters.

## Approximating a circle

Now suppose we want to draw a circle.
We can do that, approximately, by drawing a polygon with a large number of sides, so each side is small enough that it's hard to see.
Here is a function that uses `polygon` to draw a `30`-sided polygon that approximates a circle.

In [29]:
def polygon(t, length, n):
    angle = 360 / n
    for i in range(n):
        t.forward(length)
        t.left(angle)

polygon(t, 10, 30)

`circle` takes the radius of the the circle as a parameter.
It computes `circumference`, which is the circumference of a circle with the given radius.
`n` is the number of sides, so `circumference / n` is the length of each side.

This function might take a long time to run.
We can speed it up by calling `make_turtle` with a keyword argument called `delay` that sets the time, in seconds, the turtle waits after each step.
The default value is `0.2` seconds -- if we set it to `0.02` it runs about 10 times faster.

In [31]:
from jupyturtle import make_turtle

t = make_turtle(delay=0.02)

def circle(t, r):
    circumference = 2 * 3.14159 * r
    n = int(circumference / 3) + 1
    length = circumference / n
    polygon(t, length, n)

circle(t, 100)

A limitation of this solution is that `n` is a constant, which means
that for very big circles, the sides are too long, and for small
circles, we waste time drawing very short sides.
One option is to generalize the function by taking `n` as a parameter.
But let's keep it simple for now.

## Refactoring

Now let's write a more general version of `circle`, called `arc`, that takes a second parameter, `angle`, and draws an arc of a circle that spans the given angle.
For example, if `angle` is `360` degrees, it draws a complete circle. If `angle` is `180` degrees, it draws a half circle.

To write `circle`, we were able to reuse `polygon`, because a many-sided polygon is a good approximation of a circle.
But we can't use `polygon` to write `arc`.

Instead, we'll create the more general version of `polygon`, called `polyline`.

In [ ]:
def polyline(t, n, length, angle):
    for i in range(n):
        t.forward(length)
        t.left(angle)

polyline(t, 10, 10, 36)



`polyline` takes as parameters the number of line segments to draw, `n`, the length of the segments, `length`, and the angle between them, `angle`.

Now we can rewrite `polygon` to use `polyline`.

In [38]:
def polygon(t, length, n):
    angle = 360 / n
    polyline(t, n, length, angle)

polygon(t, 30, 30)


And we can use `polyline` to write `arc`.

In [39]:
def arc(t, radius, angle):
    circumference = 2 * 3.14159 * radius
    arc_length = circumference * angle / 360

    n = int(arc_length / 3) + 1
    step_length = arc_length / n
    step_angle = angle / n

    polyline(t, n, step_length, step_angle)

arc(t, 100, 180)

`arc` is similar to `circle`, except that it computes `arc_length`, which is a fraction of the circumference of a circle.

Finally, we can rewrite `circle` to use `arc`.

In [40]:
def circle(t, radius):
    arc(t, radius, 360)

circle(t, 100)

To check that these functions work as expected, we'll use them to draw something like a snail.
With `delay=0`, the turtle runs as fast as possible.

In [41]:
from jupyturtle import make_turtle

t = make_turtle(delay=0)

radius = 5

for i in range(30):
    arc(t, radius, 90)
    radius += 3

In this example, we started with working code and reorganized it with different functions.
Changes like this, which improve the code without changing its behavior, are called **refactoring**.

If we had planned ahead, we might have written `polyline` first and avoided refactoring, but often you don't know enough at the beginning of a project to design all the functions.
Once you start coding, you understand the problem better.
Sometimes refactoring is a sign that you have learned something.

## Stack diagram

When we call `circle`, it calls `arc`, which calls `polyline`.
We can use a stack diagram to show this sequence of function calls and the parameters for each one.

In [46]:
from jupyturtle import make_turtle

t = make_turtle(delay=0)

def polyline(t, n, length, angle):
    for i in range(n):
        t.forward(length)
        t.left(angle)


def arc(t, radius, angle):
    circumference = 2 * 3.14159 * radius
    arc_length = circumference * angle / 360

    n = int(arc_length / 3) + 1
    step_length = arc_length / n
    step_angle = angle / n

    polyline(t, n, step_length, step_angle)


def circle(t, radius):
    arc(t, radius, 360)

circle(t, 50)

Notice that the value of `angle` in `polyline` is different from the value of `angle` in `arc`.
Parameters are local, which means you can use the same parameter name in different functions; it's a different variable in each function, and it can refer to a different value. 

## A development plan

A **development plan** is a process for writing programs.
The process we used in this chapter is "encapsulation and generalization".
The steps of this process are:

1.  Start by writing a small program with no function definitions.

2.  Once you get the program working, identify a coherent piece of it,
    encapsulate the piece in a function and give it a name.

3.  Generalize the function by adding appropriate parameters.

4.  Repeat Steps 1 to 3 until you have a set of working functions.

5.  Look for opportunities to improve the program by refactoring. For
    example, if you have similar code in several places, consider
    factoring it into an appropriately general function.

This process has some drawbacks -- we will see alternatives later -- but it can be useful if you don't know ahead of time how to divide the program into functions.
This approach lets you design as you go along.

The design of a function has two parts:

* The **interface** is how the function is used, including its name, the parameters it takes and what the function is supposed to do.

* The **implementation** is how the function does what it's supposed to do.

For example, here's the first version of `circle` we wrote, which uses `polygon`.

In [47]:
def circle(t, radius):
    circumference = 2 * 3.14159 * radius
    n = 30
    length = circumference / n
    polygon(t, length, n)

And here's the refactored version that uses `arc`.

In [48]:
def circle(t, radius):
    arc(t, radius, 360)

These two functions have the same interface -- they take the same parameters and do the same thing -- but they have different implementations.

## Docstrings

A **docstring** is a string at the beginning of a function that explains the interface ("doc" is short for "documentation").
Here is an example:

In [ ]:
"""this is a docstring"""

'this is a docstring'

By convention, docstrings are triple-quoted strings, also known as **multiline strings** because the triple quotes allow the string to span more than one line.

A docstring should:

* Explain concisely what the function does, without getting into the details of how it works,

* Explain what effect each parameter has on the behavior of the function, and

* Indicate what type each parameter should be, if it is not obvious.

Writing this kind of documentation is an important part of interface design.
A well-designed interface should be simple to explain; if you have a hard time explaining one of your functions, maybe the interface could be improved.

## Debugging

An interface is like a contract between a function and a caller. The
caller agrees to provide certain arguments and the function agrees to
do certain work.

For example, `polyline` requires three arguments: `n` has to be an integer; `length` should be a positive number; and `angle` has to be a number, which is understood to be in degrees.

These requirements are called **preconditions** because they are supposed to be true before the function starts executing. Conversely, conditions at the end of the function are **postconditions**.
Postconditions include the intended effect of the function (like drawing line segments) and any side effects (like moving the turtle or making other changes).

Preconditions are the responsibility of the caller. If the caller violates a precondition and the function doesn't work correctly, the bug is in the caller, not the function.

If the preconditions are satisfied and the postconditions are not, the bug is in the function. If your pre- and postconditions are clear, they can help with debugging.

## Glossary

**interface design:**
A process for designing the interface of a function, which includes the parameters it should take.

**canvas:**
A window used to display graphical elements including lines, circles, rectangles, and other shapes.

**encapsulation:**
 The process of transforming a sequence of statements into a function definition.

**generalization:**
 The process of replacing something unnecessarily specific (like a number) with something appropriately general (like a variable or parameter).

**keyword argument:**
An argument that includes the name of the parameter.

**refactoring:**
 The process of modifying a working program to improve function interfaces and other qualities of the code.

**development plan:**
A process for writing programs.

**docstring:**
 A string that appears at the top of a function definition to document the function's interface.

**multiline string:**
A string enclosed in triple quotes that can span more than one line of a program.

**precondition:**
 A requirement that should be satisfied by the caller before a function starts.

**postcondition:**
 A requirement that should be satisfied by the function before it ends.

## Exercises

For the exercises below, there are a few more turtle functions you might want to use.

* `penup` lifts the turtle's imaginary pen so it doesn't leave a trail when it moves.

* `pendown` puts the pen back down.

The following function uses `penup` and `pendown` to move the turtle without leaving a trail.

In [53]:
from jupyturtle import make_turtle
t = make_turtle()

t.penup()
t.forward(100)
t.pendown()

### Exercise

Write a function called `rectangle` that draws a rectangle with given side lengths.
For example, here's a rectangle that's `80` units wide and `40` units tall.

In [54]:
def rectangle(t, width, height):
    for i in range(2):
        t.forward(width)
        t.left(90)
        t.forward(height)
        t.left(90)

rectangle(t, 80, 40)

You can use the following code to test your function.

In [55]:
from jupyturtle import make_turtle
t = make_turtle()
rectangle(t, 80, 40)

### Exercise

Write a function called `rhombus` that draws a rhombus with a given side length and a given interior angle. For example, here's a rhombus with side length `50` and an interior angle of `60` degrees.

In [58]:
def rhombus(t, size):
    for i in range(2):
        t.forward(size)
        t.left(60)
        t.forward(size)
        t.left(120)

rhombus(t,50)

You can use the following code to test your function.

In [59]:
from jupyturtle import make_turtle
t = make_turtle()
rhombus(t,50)

### Exercise

Now write a more general function called `parallelogram` that draws a quadrilateral with parallel sides. Then rewrite `rectangle` and `rhombus` to use `parallelogram`.

In [ ]:
from jupyturtle import make_turtle
t = make_turtle()
def parallelogram(t, size):
    for i in range(2):
        t.forward(size)
        t.left(60)
        t.forward(size)
        t.left(120)

parallelogram(t,50)

In [67]:
from jupyturtle import make_turtle
t = make_turtle()
def parallelogram(t, length1, length2, angle):
    for i in range(2):
        t.forward(length1)
        t.left(angle)
        t.forward(length2)
        t.left(180 - angle)


def rectangle(t, width, height):
    parallelogram(t, width, height, 90)

rectangle(t, 80, 40)

In [69]:
from jupyturtle import make_turtle
t = make_turtle()
def parallelogram(t, length1, length2, angle):
    for i in range(2):
        t.forward(length1)
        t.left(angle)
        t.forward(length2)
        t.left(180 - angle)

def rhombus(t, length, angle):
    parallelogram(t, length, length, angle)

rhombus(t,50,60)

You can use the following code to test your functions.

In [70]:
# above

### Exercise

Write an appropriately general set of functions that can draw shapes like this.

![](https://github.com/AllenDowney/ThinkPython/raw/v3/jupyturtle_pie.png)

Hint: Write a function called `triangle` that draws one triangular segment, and then a function called `draw_pie` that uses `triangle`.

In [72]:
from jupyturtle import make_turtle
t = make_turtle()

def triangle(t, length, color):
    t.forward(length)
    t.left(120)
    t.forward(length)
    t.left(120)
    t.forward(length)
    t.left(120)

def draw_pie(t, length, colors):
    for color in colors:
        triangle(t, length, color)
        t.right(360 / len(colors))


colors = ["red", "orange", "yellow", "green", "blue", "purple"]
draw_pie(t, 100, colors)

You can use the following code to test your functions.

In [74]:
from jupyturtle import make_turtle
t = make_turtle()
draw_pie(t, 100, colors)

### Exercise

Write an appropriately general set of functions that can draw flowers like this.

![](https://github.com/AllenDowney/ThinkPython/raw/v3/jupyturtle_flower.png)

Hint: Use `arc` to write a function called `petal` that draws one flower petal.

In [77]:
from jupyturtle import make_turtle
t = make_turtle(delay=0)

import math

def arc(t, r, angle):
    arc_length = 2 * math.pi * r * abs(angle) / 360
    n = int(arc_length / 2) + 1  
    step_length = arc_length / n
    step_angle = angle / n

    for _ in range(n):
        t.forward(step_length)
        t.left(step_angle)

def petal(t, r, angle):
    for _ in range(2):
        arc(t, r, angle)
        t.left(180 - angle)

def flower(t, n, r, angle):
    for _ in range(n):
        petal(t, r, angle)
        t.left(360 / n)


flower(t, n=7, r=60, angle=60)

You can use the following code to test your functions.

Because the solution draws a lot of small line segments, it tends to slow down as it runs.
To avoid that, you can add the keyword argument `auto_render=False` to avoid drawing after every step, and then call the `render` function at the end to show the result.

While you are debugging, you might want to remove `auto_render=False`.

In [78]:
auto_render=False

### Ask a virtual assistant

There are several modules like `jupyturtle` in Python, and the one we used in this chapter has been customized for this book.
So if you ask a virtual assistant for help, it won't know which module to use.
But if you give it a few examples to work with, it can probably figure it out.
For example, try this prompt and see if it can write a function that draws a spiral:

```
The following program uses a turtle graphics module to draw a circle:

from jupyturtle import make_turtle, forward, left
import math

def polygon(n, length):
    angle = 360 / n
    for i in range(n):
        forward(length)
        left(angle)
        
def circle(radius):
    circumference = 2 * math.pi * radius
    n = 30
    length = circumference / n
    polygon(n, length)
    
make_turtle(delay=0)
circle(30)

Write a function that draws a spiral.
```

Keep in mind that the result might use features we have not seen yet, and it might have errors.
Copy the code from the VA and see if you can get it working.
If you didn't get what you wanted, try modifying the prompt.


In [79]:
from jupyturtle import make_turtle, forward, left

def spiral(turns, initial_length, increment, angle):
    """
    Draw a spiral.
    
    turns: number of line segments
    initial_length: starting length of the first segment
    increment: amount to increase the segment length each step
    angle: angle to turn after each segment
    """
    length = initial_length
    for _ in range(turns):
        forward(length)
        left(angle)
        length += increment

# --- Main Program ---
make_turtle(delay=0)

# Example spiral
spiral(turns=50, initial_length=5, increment=3, angle=30)

[Think Python: 3rd Edition](https://allendowney.github.io/ThinkPython/index.html)

Copyright 2024 [Allen B. Downey](https://allendowney.com)

Code license: [MIT License](https://mit-license.org/)

Text license: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)